# Import Library

In [1]:
!pip install sklearn-genetic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from genetic_selection import GeneticSelectionCV
from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import *
from google.colab import files, drive, data_table
from google.colab.data_table import *
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Membaca data

In [4]:
# membaca data .csv
df1=pd.read_csv('/content/drive/MyDrive/DtsKominfo/Demo/Costs.csv', sep =';') 
df_price = df1['price']
df2=pd.read_csv('/content/drive/MyDrive/DtsKominfo/Demo/grocery_timestamp.csv', sep = ';') 
display(df1)

,product,demand,price,unit_cost,"order_cost,,"
0,air fresheners candles,39,"12,25","10,04",28
1,asian foods,245,"22,95","20,65",9
2,baby accessories,13,"28,92","22,27",21
3,baby bath body care,14,"24,99","20,99",5
4,baby food formula,505,"19,98","15,58",4
...,...,...,...,...,...
129,trash bags liners,66,"10,99","9,01",4
130,vitamins supplements,74,"17,89","15,21",10
131,water seltzer sparkling water,1418,"4,19","3,48",4
132,white wines,23,"13,99","10,91",13


# Exploratory Data

In [5]:
display(df2)

,order_id,pay_method,order_hour_of_day,arrival_ timestamp,pickup_timestamp,start_service_time,finish_service_time,days_since_prior_order,air fresheners candles,asian foods,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,930878.0,2.0,0.0,0:00:22,0:07:10,0:07:10,0:09:43,17.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1494544.0,2.0,0.0,0:02:45,0:07:41,0:07:41,0:10:25,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3275833.0,1.0,0.0,0:03:17,0:23:22,0:23:22,0:26:11,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,57650.0,2.0,0.0,0:03:33,0:16:29,0:16:29,0:22:11,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2002409.0,1.0,0.0,0:03:59,0:19:11,0:19:11,0:24:25,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,3193956.0,2.0,23.0,23:59:25,0:05:43,0:05:43,0:12:12,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df1.head()

,product,demand,price,unit_cost,"order_cost,,"
0,air fresheners candles,39,"12,25","10,04",28
1,asian foods,245,"22,95","20,65",9
2,baby accessories,13,"28,92","22,27",21
3,baby bath body care,14,"24,99","20,99",5
4,baby food formula,505,"19,98","15,58",4


In [7]:
df2.shape

(5004, 142)

In [8]:
df2.describe

<bound method NDFrame.describe of        order_id  pay_method  order_hour_of_day arrival_ timestamp  \
0      930878.0         2.0                0.0            0:00:22   
1     1494544.0         2.0                0.0            0:02:45   
2     3275833.0         1.0                0.0            0:03:17   
3       57650.0         2.0                0.0            0:03:33   
4     2002409.0         1.0                0.0            0:03:59   
...         ...         ...                ...                ...   
4999  3193956.0         2.0               23.0           23:59:25   
5000        NaN         NaN                NaN                NaN   
5001        NaN         NaN                NaN                NaN   
5002        NaN         NaN                NaN                NaN   
5003        NaN         NaN                NaN                NaN   

     pickup_timestamp start_service_time finish_service_time  \
0             0:07:10            0:07:10             0:09:43   
1        

# Deteksi Missing Value
missing value : data-data yang tidak terdeteksi/tidak ada data(kosong)

In [9]:
# Deteksi Adanya Missing Value data df1

print('Status Missing Values :',df1.isnull().values.any())
print('\nJumlah Missing Values masing-masing kolom, adalah:')
print(df1.isnull().sum().sort_values(ascending=False))

Status Missing Values : False

Jumlah Missing Values masing-masing kolom, adalah:
product         0
demand          0
price           0
unit_cost       0
order_cost,,    0
dtype: int64


In [10]:
# Deteksi Adanya Missing Value data df2

print('Status Missing Values :',df2.isna().values.any())
print('\nJumlah Missing Values masing-masing kolom, adalah:')
print(df2.isna().sum().sort_values(ascending=False))

Status Missing Values : True

Jumlah Missing Values masing-masing kolom, adalah:
order_id         4
oils vinegars    4
milk             4
mint gum         4
missing          4
                ..
cream            4
crackers         4
cookies cakes    4
condiments       4
yogurt           4
Length: 142, dtype: int64


In [11]:
# hapus data yang NAN
df2 = df2.dropna()
display(df2)

,order_id,pay_method,order_hour_of_day,arrival_ timestamp,pickup_timestamp,start_service_time,finish_service_time,days_since_prior_order,air fresheners candles,asian foods,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,930878.0,2.0,0.0,0:00:22,0:07:10,0:07:10,0:09:43,17.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1494544.0,2.0,0.0,0:02:45,0:07:41,0:07:41,0:10:25,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3275833.0,1.0,0.0,0:03:17,0:23:22,0:23:22,0:26:11,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,57650.0,2.0,0.0,0:03:33,0:16:29,0:16:29,0:22:11,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2002409.0,1.0,0.0,0:03:59,0:19:11,0:19:11,0:24:25,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,765994.0,3.0,23.0,23:57:33,0:01:17,0:01:17,0:04:32,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4996,1015146.0,3.0,23.0,23:58:02,0:12:59,0:12:59,0:17:41,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,2660572.0,3.0,23.0,23:58:20,0:15:04,0:15:04,0:21:18,30.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4998,563443.0,3.0,23.0,23:58:25,0:19:13,0:19:13,0:25:27,26.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


#Ambil data price dan nama product

In [12]:
df = pd.DataFrame(df1, columns=['product','price'])
price = [float(df['price'][i].replace(',','.')) for i in range(len(df))]
df['price'] = price
df

,product,price
0,air fresheners candles,12.25
1,asian foods,22.95
2,baby accessories,28.92
3,baby bath body care,24.99
4,baby food formula,19.98
...,...,...
129,trash bags liners,10.99
130,vitamins supplements,17.89
131,water seltzer sparkling water,4.19
132,white wines,13.99


In [13]:
#Hitung total price untuk tiap order dari seluruh barang 

total_price = []
for i in range(len(df2)):
   price = []
   for j in range(len(df)):
       price.append(float(df2[df['product'][j]][i]) * float(df['price'][j]))
   total_price.append(round(sum(price),3))   
print(price)
print(len(total_price),total_price)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.38, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.57, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.89]
5000 [38.78, 16.92, 198.23, 74.19, 104.66, 46.87, 180.2, 143.34, 109.53, 69.58, 90.3, 203.23, 69.69, 89.34, 110.84, 30.42, 21.47, 69.98, 27.03, 24.85, 92.99, 79.51, 82.79, 41.45, 129.18, 64.72, 41.41, 38.57, 66.79, 55.77, 54.98, 73.44, 21.97, 22.48, 110.36, 32.42, 107.99, 47.54, 37.62, 191.34, 12.48, 110.12, 106.03, 17.9

In [14]:
#Hitung time pickup dan time service

import datetime 
import time
from datetime import datetime
from datetime import timedelta

time_pickup = []
for i in range(len(df2)):
    t1 = df2['arrival_ timestamp'][i]
    t1 = datetime.strptime(t1, "%H:%M:%S")
    t2 = df2['pickup_timestamp'][i]
    t2 = datetime.strptime(t2, "%H:%M:%S")
    if(t1<t2): t = t2-t1
    elif(t1>t2):
        t = t2-t1
        t = t + timedelta(days = 1)
    else: t = 0
    
    time_pickup.append(str(t))
print(time_pickup)

time_service = []
for i in range(len(df2)):
    t1 = df2['start_service_time'][i]
    t1 = datetime.strptime(t1, "%H:%M:%S")
    t2 = df2['finish_service_time'][i]
    t2 = datetime.strptime(t2, "%H:%M:%S")
    if(t1<t2): t = t2-t1
    elif(t1>t2):
        t = t2-t1
        t = t + timedelta(days = 1)
    else: t = 0
    
    time_service.append(str(t))
print(time_service)

['0:06:48', '0:04:56', '0:20:05', '0:12:56', '0:15:12', '0:05:44', '0:24:24', '0:21:23', '0:12:28', '0:08:21', '0:15:23', '0:25:06', '0:09:36', '0:11:26', '0:08:02', '0:07:08', '0:04:16', '0:09:55', '0:04:49', '0:04:59', '0:12:20', '0:10:00', '0:13:19', '0:07:11', '0:17:03', '0:10:28', '0:06:39', '0:07:06', '0:04:59', '0:07:55', '0:08:42', '0:11:39', '0:03:45', '0:03:35', '0:10:56', '0:05:53', '0:14:24', '0:03:43', '0:07:21', '0:20:44', '0:03:49', '0:13:23', '0:16:07', '0:03:52', '0:11:44', '0:15:35', '0:07:22', '0:08:30', '0:09:02', '0:04:52', '0:04:12', '0:06:29', '0:17:12', '0:10:46', '0:05:52', '0:04:39', '0:04:45', '0:44:55', '0:07:38', '0:00:43', '0:10:24', '0:10:57', '0:06:19', '0:14:13', '0:05:48', '0:04:30', '0:09:41', '0:05:31', '0:04:09', '0:02:44', '0:14:51', '0:19:02', '0:07:08', '0:05:58', '0:19:05', '0:04:07', '0:03:26', '0:18:02', '0:11:05', '0:21:58', '0:23:12', '0:13:39', '0:06:45', '0:02:35', '0:08:01', '0:16:23', '0:16:09', '0:07:43', '0:04:07', '0:11:21', '0:23:46'

In [15]:
# Buat Kolom di dataframe untuk lama waktu pickup, service dan total harga
df2['time_pickup'] = time_pickup
df2['time_service'] = time_service
df2['Total_Price'] = total_price

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [16]:
# Simpan data hasil bentukan ke file agar bisa di import ulang lain waktu
df2.to_csv('/content/drive/MyDrive/DtsKominfo/Demo/Data_Demo_Auto_6_v1.csv', index=False, header=True)

In [17]:
# membaca Ulang data Data_Demo_Auto_6.xlsx yang telah digabungkan dalam file excell
excel = '/content/drive/MyDrive/DtsKominfo/Demo/Data_Demo_Auto_6.xlsx'
df1 = pd.read_excel(excel, 'Costs')
df2 = pd.read_excel(excel, 'Data_Demo_Auto_6')

# membaca Ulang data Data_Demo_Auto_6.xlsx yang telah digabungkan dalam file csv
#df2=pd.read_csv('/content/drive/MyDrive/DtsKominfo/Demo/Data_Demo_Auto_6.csv') 
display(df2)

,order_id,pay_method,order_hour_of_day,arrival_ timestamp,pickup_timestamp,start_service_time,finish_service_time,days_since_prior_order,air fresheners candles,asian foods,...,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt,time_pickup,time_service,Total_Price
0,930878,2,0,00:00:22,00:07:10,00:07:10,00:09:43,17,0,0,...,0,0,0,0,0,0,0,00:06:48,00:02:33,38.78
1,1494544,2,0,00:02:45,00:07:41,00:07:41,00:10:25,11,0,0,...,0,0,0,0,0,0,0,00:04:56,00:02:44,16.92
2,3275833,1,0,00:03:17,00:23:22,00:23:22,00:26:11,3,0,0,...,0,0,0,0,0,0,1,00:20:05,00:02:49,198.23
3,57650,2,0,00:03:33,00:16:29,00:16:29,00:22:11,3,0,0,...,0,0,0,0,1,0,0,00:12:56,00:05:42,74.19
4,2002409,1,0,00:03:59,00:19:11,00:19:11,00:24:25,10,0,0,...,0,0,0,0,0,0,1,00:15:12,00:05:14,104.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,765994,3,23,23:57:33,00:01:17,00:01:17,00:04:32,13,0,0,...,0,0,1,0,0,0,0,00:03:44,00:03:15,16.97
4996,1015146,3,23,23:58:02,00:12:59,00:12:59,00:17:41,19,0,0,...,0,0,0,0,0,0,0,00:14:57,00:04:42,128.54
4997,2660572,3,23,23:58:20,00:15:04,00:15:04,00:21:18,30,0,0,...,0,1,0,0,0,0,1,00:16:44,00:06:14,113.75
4998,563443,3,23,23:58:25,00:19:13,00:19:13,00:25:27,26,0,0,...,0,0,0,0,0,0,3,00:20:48,00:06:14,145.67


In [18]:
# Ambil data untuk pembentukan input 
df = pd.DataFrame(df1, columns=['product','price'])
price = [float(df['price'][i].replace(',','.')) for i in range(len(df))]
df['price'] = price
df

,product,price
0,air fresheners candles,12.25
1,asian foods,22.95
2,baby accessories,28.92
3,baby bath body care,24.99
4,baby food formula,19.98
...,...,...
129,trash bags liners,10.99
130,vitamins supplements,17.89
131,water seltzer sparkling water,4.19
132,white wines,13.99


# Menentukan data inputan dan Output

In [19]:
# Menentukan data inputan (seluruh barang tiap order)
X = df2.drop(columns=['order_id', 'pay_method', 'order_hour_of_day', 'arrival_ timestamp', 'pickup_timestamp', 'start_service_time', 'finish_service_time', 'days_since_prior_order','time_pickup', 'time_service', 'Total_Price'])
X

,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4996,0,0,0,0,0,0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
4998,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,3


In [20]:
# Rubah tipe data dalam Input  X ke float dari Int64
for j in range(len(df)):
    X[df['product'][j]] = X[df['product'][j]].astype(float)

In [21]:
#buat data input, jumlah barang x price

for j in range(len(df)):
    for i in range(len(X)):      
        X[df['product'][j]][i] = float(X[df['product'][j]][i] * df['price'][j])
display(X)

,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00
2,0.0,0.0,0.0,0.0,0.0,15.99,43.96,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,3.89
3,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,4.19,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,10.99,0.0,0.00,0.0,0.00
4996,0.0,0.0,0.0,0.0,0.0,0.00,32.97,10.8,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00
4997,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,4.19,0.00,0.00,0.0,5.99,0.00,0.0,0.00,0.0,3.89
4998,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,4.19,3.49,2.69,0.0,0.00,0.00,0.0,0.00,0.0,11.67


In [22]:
# Simpan hasil bentukan Data input ke file 
X.to_csv('/content/drive/MyDrive/DtsKominfo/Demo/X_auto_6.csv', index=False, header=True)

In [23]:
# Menentukan data output atau target
Y = pd.DataFrame(df2['Total_Price'])
Y

,Total_Price
0,38.78
1,16.92
2,198.23
3,74.19
4,104.66
...,...
4995,16.97
4996,128.54
4997,113.75
4998,145.67


# Feature Selection 

In [24]:
# Fitur seleksi dengan genetic Algorithm estimator Decission Tree
estimators = DecisionTreeRegressor()

t1=time.time()
print(t1)

report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 

print("Estimator dipakai : ", estimators )
 
selector = GeneticSelectionCV(estimators,
                                cv = 5,
                                verbose = 1,
                                scoring="neg_mean_squared_error", 
                                max_features = 30,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 30,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.1,
                                tournament_size = 3,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
selector = selector.fit(X, Y)
genfeats = X.columns[selector.support_]
genfeats = list(genfeats)
print("Chosen Feats: {} of {}, scores : {} " .format(genfeats, selector.n_features_, round(selector.generation_scores_[-1], 3)))

cv_score = selector.generation_scores_[-1]
nofeats.append(len(genfeats)) 
chosen_feats.append(genfeats) 
cvscore.append(cv_score)

report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore
#Lama waktu Proses 
t2=time.time()
t_polyfit = float(t2-t1)
print("Lama waktu Proses fitur seleksi : {} seconds".format(t_polyfit))

1657110556.5761085
Estimator dipakai :  DecisionTreeRegressor()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Selecting features with genetic algorithm.
gen	nevals	avg                                     	std                                  	min                                     	max                                     
0  	200   	[-2682.971969    15.985      247.67503 ]	[ 339.958925    8.560069   77.451798]	[-3303.85048      1.          37.755488]	[-1851.870052    30.         559.421966]
1  	110   	[-3101.825213    21.53      1157.273344]	[ 2248.579329     7.479245  2865.970756]	[-10000.            5.           37.755488]	[-1715.695755    38.       10000.      ]
2  	127   	[-3267.377739    23.52      1546.852906]	[ 2669.943638     6.853437  3340.623169]	[-10000.            7.           37.755488]	[-1715.695755    41.       10000.      ]
3  	117   	[-3065.805139    24.185     1430.007353]	[ 2629.169765     6.240254  3240.136174]	[-10000.            7.           37.755488]	[-1629.225441    43.       10000.      ]
4  	118   	[-3117.526917    25.05      1600.24585 ]	[ 2840.209714     6.207052 

In [25]:
#Print Reports hasil fitur seleksi
report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = True)
#report.index
ga_feats = report.iloc[0]["Chosen Feats"]
DataTable(report)
print("Feature selection recommend:", ga_feats)
print("Estimator : {}, reports : ". format(selector.estimator_))
report

Feature selection recommend: ['asian foods', 'baby accessories', 'baby food formula', 'beauty', 'beers coolers', 'butter', 'canned jarred vegetables', 'cereal', 'chips pretzels', 'cleaning products', 'cookies cakes', 'crackers', 'food storage', 'fresh fruits', 'fresh vegetables', 'frozen meals', 'kitchen supplies', 'meat counter', 'mint gum', 'oils vinegars', 'packaged cheese', 'packaged seafood', 'packaged vegetables fruits', 'plates bowls cups flatware']
Estimator : DecisionTreeRegressor(), reports : 


,No of Feats,Chosen Feats,Scores
0,24,"[asian foods, baby accessories, baby food form...",-1278.61


In [26]:
# Cetak Hasil Fitur terseleksi yang pengaruhnya besar terhadap penjualan
np.transpose(ga_feats).tolist()


['asian foods',
 'baby accessories',
 'baby food formula',
 'beauty',
 'beers coolers',
 'butter',
 'canned jarred vegetables',
 'cereal',
 'chips pretzels',
 'cleaning products',
 'cookies cakes',
 'crackers',
 'food storage',
 'fresh fruits',
 'fresh vegetables',
 'frozen meals',
 'kitchen supplies',
 'meat counter',
 'mint gum',
 'oils vinegars',
 'packaged cheese',
 'packaged seafood',
 'packaged vegetables fruits',
 'plates bowls cups flatware']